In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

In [ ]:
orders_data = pd.read_csv('/content/orders.csv')

In [ ]:
orders_data.head()

In [ ]:
orders_data.shape

In [ ]:
visitor_email_data = pd.read_csv("/content/smatlook_visitor_vs_email.csv")

In [ ]:
visitor_email_data.head()

In [ ]:
visitor_email_data.shape

In [ ]:
sessions_data = pd.read_csv("/content/smartlook_session.csv")

In [ ]:
sessions_data.head()

In [ ]:
sessions_data.shape

In [ ]:
questionnaire_data = pd.read_csv("/content/questionnaire_cleaned.csv")

In [ ]:
questionnaire_data.head()

In [ ]:
questionnaire_data.shape

In [ ]:
master_df = orders_data.merge(visitor_email_data, on='email_hash', how='left')

In [ ]:
master_df.head()

In [ ]:
sessions_data['startedAt'] = pd.to_datetime(sessions_data['startedAt'])
sessions_data['endedAt'] = pd.to_datetime(sessions_data['endedAt'])

In [ ]:
sessions_data['session_duration'] = (sessions_data['endedAt'] - sessions_data['startedAt']).dt.total_seconds()

In [ ]:
session_agg = sessions_data.groupby('visitorId').agg({
    'session_duration' : 'mean',
    'referrer' : 'first',
    'device_type' : lambda x: x.mode().iloc[0] if not x.mode().empty else 'unknown'}).reset_index()

In [ ]:
session_count = sessions_data.groupby('visitorId').size().reset_index(name='total_session')

In [ ]:
sessions_agg = session_agg.merge(session_count, on='visitorId')

In [ ]:
sessions_agg.columns

In [ ]:
sessions_agg = sessions_agg[['visitorId'  ,'total_session', 'session_duration' , 'referrer' , 'device_type']]

In [ ]:
sessions_agg.columns = ['visitorId' , 'total_sessions' , 'avg_session_duration' , 'first_referrer' , 'most_common_device']

In [ ]:
sessions_agg.shape

In [ ]:
master_df = master_df.merge(sessions_agg, on='visitorId', how='left')

In [ ]:
master_df.head(50)

In [ ]:
parent_orders = master_df[master_df['is_parent_order'] == 1]
questionnaire_merged = parent_orders.merge(questionnaire_data, left_on='orderId', right_on = 'order_id', how='left')

In [ ]:
questionnaire_customers = questionnaire_merged[['email_hash' ] + list(questionnaire_data.columns)].drop_duplicates('email_hash')

In [ ]:
master_df = master_df.merge(questionnaire_customers , on= 'email_hash' , how = 'left')

In [ ]:
ltv_df = orders_data.groupby('email_hash')['total'].sum().reset_index()
ltv_df.columns = ['email_hash' , 'LTV']

In [ ]:
customer_metrics = orders_data.groupby('email_hash').agg({
    'orderId' : 'count' ,
    'total' : ['sum' , 'mean'] ,
    'is_parent_order' : lambda x: (x == 0).sum(),
    'cart_discount' : 'sum'

}).reset_index()

In [ ]:
customer_metrics.columns = ['email_hash' , 'total_orders' , 'LTV' , 'avg_order_value' , 'renewal_count' , 'total_discount']

In [ ]:
current_year = 2025
if 'birth_year' in master_df.columns:
    master_df['age'] = current_year - master_df['birth_year']

In [ ]:
def extract_aquisition_source(referrer):
  if pd.isna(referrer) or referrer == '':
    return 'direct'
  referrer = str(referrer).lower()
  if 'google' in referrer:
    return 'google'
  elif 'facebook' in referrer:
    return 'facebook'
  elif 'bing' in referrer:
    return 'bing'
  elif 'instagram' in referrer:
    return 'instagram'
  elif 'email' in referrer:
    return 'email'
  else:
    return "other"

In [ ]:
master_df['acquisition_source'] = master_df['first_referrer'].apply(extract_aquisition_source)

In [ ]:
def extract_utm (url , param):
  if pd.isna(url):
    return 'unknown'
  try:
    if f'{param}=' in str(url):
      start = str(url).find(f'{param}=') + len(f'{param}=')
      end = str(url).find('&',start)
      if end == -1:
        return str(url)[start:]
      return str(url)[start:end]
  except:
    pass
  return 'unknown'


In [ ]:
if 'landingPage' in sessions_data.columns:
  utm_data = sessions_data.groupby('visitorId')['landingPage'].first().reset_index()
  utm_data['utm_source'] = utm_data['landingPage'].apply(lambda x: extract_utm(x , 'utm_source'))
  utm_data['utm_medium'] = utm_data['landingPage'].apply(lambda x: extract_utm(x , 'utm_medium'))
  utm_data['utm_campaign'] = utm_data['landingPage'].apply(lambda x: extract_utm(x , 'utm_campaign'))
  master_df = master_df.merge(utm_data[['visitorId' , 'utm_source' , 'utm_medium' , 'utm_campaign']], on = 'visitorId' , how = 'left')

In [ ]:
customer_df = master_df.groupby('email_hash').agg({
    'visitorId' : 'first',
    'total_sessions' : 'first',
    'avg_session_duration' : 'first',
    'acquisition_source' : 'first',
    'most_common_device' : 'first',
    'age' : 'first',
    'shipping_state' : 'first'
}).reset_index()

In [ ]:
if 'utm_source' in master_df.columns:
  utm_customer = master_df.groupby('email_hash')[['utm_source' , 'utm_medium' , 'utm_campaign']].first().reset_index()
  customer_df = customer_df.merge(utm_customer, on='email_hash', how='left')


In [ ]:
questionnaire_cols = [col for col in questionnaire_data.columns if col != 'orderID']

In [ ]:
if questionnaire_cols:
  quest_customer = master_df.groupby('email_hash')[questionnaire_cols].first().reset_index()
  customer_df = customer_df.merge(quest_customer, on='email_hash', how='left')

In [ ]:
customer_df = customer_df.merge(customer_metrics, on='email_hash', how='left')


In [ ]:
customer_df.head()

In [ ]:
ltv_by_channel = customer_df.groupby('acquisition_source')['LTV'].agg(['mean' , 'count']).round(2)
ltv_by_channel.columns = ['Avg_LTV' , 'Cusotomer_Count']
print("Acquisition Channel\n\n\n" , ltv_by_channel.sort_values('Avg_LTV' , ascending = False))

In [ ]:
ltv_by_channel = customer_df.groupby('most_common_device')['LTV'].agg(['mean' , 'count']).round(2)
ltv_by_channel.columns = ['Avg_LTV' , 'Cusotomer_Count']
print("Device Type Channel\n\n\n" , ltv_by_channel.sort_values('Avg_LTV' , ascending = False))

In [ ]:
if 'total_sessions' in customer_df.columns:
  sessions_ltv_corr = customer_df[['total_sessions' , 'avg_session_duration' , 'LTV']].corr()
  print('\n Correlation with LTV')
  print(sessions_ltv_corr['LTV'].sort_values(ascending = False))

In [ ]:
model_df = customer_df.copy()
feature_columns = ['age' , 'total_sessions' , 'avg_session_duration' , 'total_orders' , 'renewal_count']

In [ ]:
categorical_features = ['acquisition_source' , 'most_common_device' , 'shipping_state']

In [ ]:
if 'utm_source' in model_df.columns:
  categorical_features.extend(['utm_source' , 'utm_medium' , 'utm_campaign'])

In [ ]:
label_encoders =  {}
for col in categorical_features:
  if col in model_df.columns:
    encoder = LabelEncoder()
    model_df [col + '_encoded'] = encoder.fit_transform(model_df[col].fillna('unknown'))
    feature_columns.append(col + '_encoded')
    label_encoders[col] = encoder

In [ ]:
for col in questionnaire_data.columns:
    if col in model_df.columns and col not in ['orderId', 'order_id']:
        if model_df[col].isna().all():
          print(f"Skipping {col} - all values are NaN")
          continue

        print(f"Processing column: {col}")
        print(f"Original data types in {col}: {model_df[col].apply(type).value_counts()}")
        column_data = model_df[col].fillna('unknown').astype(str)

        le = LabelEncoder()
        encoded_col_name = col + '_encoded'

        try:
            model_df[encoded_col_name] = le.fit_transform(column_data)
            feature_columns.append(encoded_col_name)
            label_encoders[col] = le
            questionnaire_binary_cols.append(encoded_col_name)

            print(f"Successfully encoded {col} -> {encoded_col_name}")
            print(f"Unique values: {le.classes_}")

        except Exception as e:
            print(f"Error encoding {col}: {str(e)}")
            print(f"Sample values: {column_data.unique()[:5]}")
            continue


In [ ]:
model_features = model_df[feature_columns].fillna(0)
target = model_df['LTV'].fillna(0)

In [ ]:
valid_customers = target > 0
X = model_features [valid_customers]
y = target[valid_customers]

In [ ]:
print(f"Training dataset: {X.shape[0]} customers, {X.shape[1]} features")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42 ,max_depth  =10)
model.fit(X_train, y_train)
y_pred= model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


In [ ]:
print(f"Model Performance:")
print(f"Mean Absolute Error: {mae}")
print(f"R2 Score: {r2}")

In [ ]:
feature_importance = pd.DataFrame({
    'feature'  : X.columns,
    'important' : model.feature_importances_
}).sort_values('important' , ascending = False)

In [ ]:
print (f"Top most important features for customer life time value: {feature_importance.head()}" )

EDA

In [ ]:
def create_customer_segment (row):
  ltv = row['LTV']
  renewals = row['renewal_count']
  orders = row['total_orders']

  if ltv > customer_df['LTV'].quantile(0.75) and renewals > 2:
    return "High-Value Renewers"
  elif orders ==1 and renewals ==0:
    return 'One-and-Dones'
  elif row['total_discount'] > customer_df['total_discount'].quantile(0.75):
    return "Bargain Hunters"
  elif ltv > customer_df['LTV'].median():
    return 'Solid Customers'
  else:
    return "Low-Value Customers"
customer_df['segment'] = customer_df.apply(create_customer_segment, axis=1)

In [ ]:
segment_analysis = customer_df.groupby('segment').agg({
    'LTV' : ['count' , 'mean' ],
    'total_sessions' : 'mean',
    'renewal_count' : 'mean',
    'total_discount' : 'mean'}).round(2)

In [ ]:
print(f'Customer Segments Analysis:{segment_analysis}')

In [ ]:
facts_and_insights = []

In [ ]:
mobile_ltv  =customer_df[customer_df['most_common_device'] == 'mobile']['LTV'].mean()
desktop_ltv  =customer_df[customer_df['most_common_device'] == 'desktop']['LTV'].mean()

In [ ]:
if desktop_ltv > mobile_ltv :
  difference = ((desktop_ltv - mobile_ltv) / mobile_ltv) * 100
  fact1 = f'Desktop users have {difference}% higher LTV than mobile users ({desktop_ltv} vs {mobile_ltv})'
  facts_and_insights.append(("Device Impact" , fact1))

In [ ]:
best_channel = ltv_by_channel.idxmax()['Avg_LTV']
best_channel_ltv = ltv_by_channel.loc[best_channel , 'Avg_LTV']
worst_channel = ltv_by_channel.idxmin()['Avg_LTV']
worst_channel_ltv = ltv_by_channel.loc[worst_channel , 'Avg_LTV']

In [ ]:
fact2 = f"customer from {best_channel} have the highest LTV ({best_channel_ltv}) , while {worst_channel} has the lowest ({worst_channel_ltv})"
facts_and_insights.append(('Acquisition Channels' , fact2))

In [ ]:
if 'total_sessions' in customer_df.columns:
  high_sessions = customer_df[customer_df['total_sessions'] > customer_df['total_sessions'].quantile(0.75)]
  low_sessions = customer_df[customer_df['total_sessions'] > customer_df['total_sessions'].quantile(0.25)]


  high_sessions_ltv = high_sessions['LTV'].mean()
  low_sessions_ltv = low_sessions['LTV'].mean()

  if high_sessions_ltv > low_sessions_ltv:
    difference = ((high_sessions_ltv - low_sessions_ltv) / low_sessions_ltv) * 100
    fact3 = f'High-engagement customers (top 25% by sessions) have  {difference}% higher LTV ({high_sessions_ltv} vs {low_sessions_ltv})'
    facts_and_insights.append(('Website Engagement' , fact3))

In [ ]:
for category, fact in facts_and_insights:
    print(f"\n{category}: {fact}")

In [ ]:
insights = [
    {
        "title": "Optimize for Desktop Experience",
        "finding": f"Desktop users show {((desktop_ltv - mobile_ltv) / mobile_ltv) * 100:.1f}% higher LTV",
        "action": "Invest in desktop UX improvements and consider desktop-specific marketing campaigns"
    },
    {
        "title": "Focus Marketing Budget on Top Channels",
        "finding": f"{best_channel} acquisition shows highest customer value",
        "action": f"Increase marketing budget allocation to {best_channel} and analyze what makes these customers more valuable"
    },
    {
        "title": "Engage High-Value Segments",
        "finding": f"{customer_df[customer_df['segment'] == 'High-Value Renewers'].shape[0]} customers in 'High-Value Renewers' segment",
        "action": "Create VIP program and personalized retention campaigns for this segment"
    },
    {
        "title": "Reduce One-and-Done Churn",
        "finding": f"{customer_df[customer_df['segment'] == 'One-and-Dones'].shape[0]} customers made only one purchase",
        "action": "Implement targeted email sequences and first-purchase follow-up campaigns"
    }
]

In [ ]:
for i , insight in enumerate(insights, 1):
  print(f"{i}. {insight['title']}")
  print(f"Finding: {insight['finding']}")
  print(f"Action: {insight['action']}")

In [ ]:
customer_df.to_csv("customer_analysis_results.csv" , index = False)
feature_importance.to_csv("feature_importance.csv" , index = False)
master_df.to_csv("master_df.csv" , index = False)